In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import librosa
import glob
import sys

from keras.callbacks import CSVLogger
from keras.optimizers import Adam

sys.path.insert(0,'..')
from mavd.model import build_custom_cnn
from mavd.callbacks import MetricsCallback
from mavd.data_generator import DataGenerator

os.environ["CUDA_VISIBLE_DEVICES"]="0"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# files parameters
Nfiles = None
resume = False
load_subset = Nfiles

# audio parameters
sr = 22050
sequence_time = 1.0
sequence_hop_time = 1.0
audio_hop = 512
audio_win = 1024
n_fft = 1024
normalize_data = 'none' # para comparar loss con MST
get_annotations = True
mel_bands = 128
htk = True
normalize_energy = True

# training
learning_rate = 0.001
epochs = 101
batch_size = 64
sed_early_stopping = 100
epoch_limit = None
fit_verbose = True
fine_tuning = False

#model
large_cnn = True
frames = True

label_list = (['air_conditioner', 'car_horn', 'children_playing',
               'dog_bark', 'drilling', 'engine_idling', 'gun_shot',
               'jackhammer', 'siren', 'street_music'])  

 # Create output folders
expfolder = '../exps/S-CNN_baseline/'

audio_folder = '/data_ssd/users/pzinemanas/maestria/URBAN-SED/audio22050'
label_folder='/data_ssd/users/pzinemanas/maestria/URBAN-SED/annotations'

In [11]:
params = {'files_batch':20, 'path':audio_folder, 'sequence_time': sequence_time, 
          'sequence_hop_time':sequence_hop_time,'label_list':label_list,
          'normalize_energy':normalize_energy,'audio_hop':audio_hop, 'audio_win':audio_win,
          'n_fft':n_fft,'sr':sr,'mel_bands':mel_bands,'normalize':normalize_data, 'frames':frames,
          'get_annotations':get_annotations, 'dataset': 'URBAN-SED'}

sequence_frames = int(np.ceil(sequence_time*sr/audio_hop))

# Datasets
partition = {}# IDs
labels = {}# Labels

train_files = sorted(glob.glob(os.path.join(audio_folder,'train', '*.wav')))
val_files = sorted(glob.glob(os.path.join(audio_folder,'validate', '*.wav')))

if load_subset is not None:
    train_files = train_files[:load_subset]
    val_files = val_files[:load_subset]

train_labels = {}
train_mel = {}
val_labels = {}
val_mel = {}
mel_basis = librosa.filters.mel(sr,n_fft,mel_bands,htk=True)
print('Founding scaler')
for n,id in enumerate(train_files):
    labels[id] = os.path.join(label_folder, 'train',os.path.basename(id).replace('.wav','.txt'))
    #train_mel[id]  = os.path.join(mel_folder, 'train',os.path.basename(id).replace('.wav','.npy.gz'))
for id in val_files:
    labels[id] = os.path.join(label_folder, 'validate',os.path.basename(id).replace('.wav','.txt'))

params['train'] = True
# Generators
print('Making generators')
training_generator = DataGenerator(train_files, labels, **params)
scaler = training_generator.get_scaler()
#print('scaler',scaler)

params['scaler'] = scaler
params['train'] = False
params['sequence_hop_time'] = sequence_time

validation_generator = DataGenerator(val_files, labels, **params)

print('Getting data')

train_example,_,train_mel_example,train_y_example = training_generator.return_random()
_,_,val_example,val_y_example = validation_generator.return_random()

_,_,x_val,y_val = validation_generator.return_all()
_,_,x_train,y_train = training_generator.return_all()

if not frames:    
    x_train = np.transpose(x_train,(0,2,1))    
    x_val = np.transpose(x_val,(0,2,1))  
    val_example = np.transpose(val_example,(0,2,1))     


print(x_train.shape)    
print(x_val.shape)     

print(x_val.shape,y_val[0].shape,y_val[1].shape)
sequence_frames = x_val.shape[1]
standard_scaler = training_generator.get_standard_scaler()

mean= standard_scaler.mean_
scale = standard_scaler.scale_

# Build model

print('\nBuilding model...')

sequence_samples = int(sequence_time*sr)

model = build_custom_cnn(n_freq_cnn=mel_bands, n_frames_cnn=sequence_frames,large_cnn=large_cnn)

##### Inicializo
model.layers[1].set_weights([np.ones_like(mean),np.zeros_like(mean),mean,scale])

model.summary()

opt = Adam(lr=learning_rate)

if resume:
    print('Loading best weights and resuming...')
    weights_best_file = os.path.join(expfolder, 'weights_best.hdf5')
    model.load_weights(weights_best_file)

# Fit model
print('\nFitting model...')

if resume:
    f1s_best = resume_f1_best

metrics_callback = MetricsCallback(x_val, y_val, 0, 0, os.path.join(expfolder, 'weights_best.hdf5'))
#save_fig = SaveFigCallback(train_example,train_y_example,val_example,val_y_example,'./mels/')
csv_logger = CSVLogger(os.path.join(expfolder, 'training.log'))

model.compile(loss='binary_crossentropy',optimizer=opt)

history = model.fit(x=x_train, y=y_train, batch_size=2*batch_size, #Borrar el 10!
                            epochs=epochs, verbose=fit_verbose,
                            validation_split=0.0,
                            shuffle=True,
                            #callbacks=[MyCallback(alpha, beta),metrics_callback,save_fig])
                            callbacks=[metrics_callback,csv_logger])


Founding scaler
Making generators
Getting data
(440, 128)
(2640000, 128)


W0829 13:25:36.044090 140142706325248 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



(60000, 44, 128)
(20000, 44, 128)
(20000, 44, 128) (10,) (10,)

Building model...


W0829 13:25:36.527404 140142706325248 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 13:25:36.802807 140142706325248 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0829 13:25:36.851987 140142706325248 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 13:25:36.895498 140142706325248 deprecation_wrapper.py:119] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 44, 128)           0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 44, 128)           512       
_________________________________________________________________
lambda_1 (Lambda)            (None, 44, 128, 1)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 124, 128)      3328      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 62, 128)       0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 20, 62, 128)       512       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 58, 128)       409728    
__________

W0829 13:25:48.014041 140142706325248 deprecation.py:323] From /home/pzinemanas/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/101
60000/60000 [==============================] - 31s 521us/step - loss: 0.3811
F1 = 0.0027, ER = 0.9987 -  Best val F1s: 0.0027 (IMPROVEMENT, saving)

Epoch 2/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.3434
F1 = 0.0000, ER = 1.0000 - Best val F1s: 0.0027 (0)

Epoch 3/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.3389
F1 = 0.0098, ER = 0.9951 -  Best val F1s: 0.0098 (IMPROVEMENT, saving)

Epoch 4/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.3364
F1 = 0.0371, ER = 0.9809 -  Best val F1s: 0.0371 (IMPROVEMENT, saving)

Epoch 5/101
60000/60000 [==============================] - 24s 404us/step - loss: 0.3353
F1 = 0.0212, ER = 0.9892 - Best val F1s: 0.0371 (3)

Epoch 6/101
60000/60000 [==============================] - 24s 404us/step - loss: 0.3344
F1 = 0.0195, ER = 0.9902 - Best val F1s: 0.0371 (3)

Epoch 7/101
60000/60000 [==============================] - 24s 404us/step - loss: 0.3319
F1

60000/60000 [==============================] - 24s 403us/step - loss: 0.1884
F1 = 0.5183, ER = 0.5913 -  Best val F1s: 0.5183 (IMPROVEMENT, saving)

Epoch 55/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.1865
F1 = 0.5167, ER = 0.5898 - Best val F1s: 0.5183 (53)

Epoch 56/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.1852
F1 = 0.5195, ER = 0.5870 -  Best val F1s: 0.5195 (IMPROVEMENT, saving)

Epoch 57/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.1832
F1 = 0.5128, ER = 0.5964 - Best val F1s: 0.5195 (55)

Epoch 58/101
60000/60000 [==============================] - 24s 404us/step - loss: 0.1824
F1 = 0.5224, ER = 0.5820 -  Best val F1s: 0.5224 (IMPROVEMENT, saving)

Epoch 59/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.1818
F1 = 0.5201, ER = 0.5880 - Best val F1s: 0.5224 (57)

Epoch 60/101
60000/60000 [==============================] - 24s 403us/step - loss: 0.1801
F1 = 